In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
!nvidia-smi

Sat Nov  7 02:28:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    29W /  70W |    227MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
def normalize(data):
#     tensor = []
    for i in range(len(data)):
        data[i] = MinMaxScaler().fit_transform(data[i])
        data[i] = data[i].astype('float32')
    return data

In [6]:
npy_path = '/content/drive/My Drive/project/smart_manufacturing/npy_file'
csv_path = '/content/drive/My Drive/project/smart_manufacturing/csv_file'
train = pd.read_csv(os.path.join(csv_path, 'encode_train_features.csv'))
train_log = np.load(os.path.join(npy_path, 'train_log_array.npy'), allow_pickle = True)
train_log = normalize(train_log)
train = MinMaxScaler().fit_transform(train)
target = np.load(os.path.join(npy_path, 'target_array.npy'))
# train_log = np.load(os.path.join(path, 'train_log_array.npy'), allow_pickle = False)
# test = np.load(os.path.join(path, 'test_array.npy'))
# test_log = np.load(os.path.join(path, 'test_log_array.npy'), allow_pickle = True)

In [9]:
type(train_log)

numpy.ndarray

In [10]:
def CNN_ANN(data_1, data_2):
#     input_ann = tf.keras.Input(shape = (1, 64))
    input_ann = tf.keras.Input(shape = data_1.shape[1:])
    input_cnn = tf.keras.Input(shape = data_2.shape[1:])
    
    ann = tf.keras.layers.Dense(16, activation = 'relu')(input_ann)
    ann = tf.keras.layers.Dense(32, activation = 'relu')(ann)
    ann = tf.keras.layers.Flatten()(ann)
    
    cnn = tf.keras.layers.Conv1D(16, kernel_size = 5, padding = 'same', activation = 'relu')(input_cnn)
    cnn = tf.keras.layers.MaxPool1D(pool_size = 5, padding = 'same')(cnn)
    cnn = tf.keras.layers.Conv1D(32, kernel_size = 5, padding = 'same', activation = 'relu')(cnn)
    cnn = tf.keras.layers.MaxPool1D(pool_size = 5, padding = 'same')(cnn)
    cnn = tf.keras.layers.Flatten()(cnn)
    cnn = tf.keras.layers.Dense(16, activation = 'relu')(cnn)
    cnn = tf.keras.layers.Dense(16, activation = 'relu')(cnn)
    
    combine = tf.keras.layers.concatenate([ann, cnn])
    
    output = tf.keras.layers.Dense(4, activation = 'linear')(combine)
    model = tf.keras.Model([input_ann, input_cnn], output)
    model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'MAE', metrics = ['MeanAbsoluteError'])
    
    return model

In [11]:
CNN_ANN(train, train_log).summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 292, 146)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 292, 16)      11696       input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 59, 16)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 59, 32)       2592        max_pooling1d[0][0]              
_______________________________________________________________________________________

In [ ]:
def ANN():
    
    input_data = tf.keras.Input(shape = (1, 64))
    
    x = tf.keras.layers.Dense(8, activation = 'relu')(input_data)
    
    x = tf.keras.layers.Dense(16, activation = 'relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    model = tf.keras.Model(input_data, x)
    
    return model

In [ ]:
def CNN():
    input_log = tf.keras.Input(shape = (292, 145))
    x = tf.keras.layers.Conv1D(8, kernel_size = 5, padding = 'same', activation = 'relu')(input_log)
    
    x = tf.keras.layers.MaxPool1D(pool_size = 5, padding = 'same')(x)
    
    x = tf.keras.layers.Conv1D(16, kernel_size = 5, padding = 'same', activation = 'relu')(x)
    
    x = tf.keras.layers.MaxPool1D(pool_size = 5, padding = 'same')(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(8, activation = 'relu')(x)
    
    x = tf.keras.layers.Dense(8, activation = 'relu')(x)
    
    model = tf.keras.Model(input_log, x)
    
#     model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'MAE', metrics = ['MeanAbsoluteError'])
    
    return model

In [12]:
model = CNN_ANN(train, train_log)
# model.summary()
model.fit([train, train_log], target, epochs = 50, batch_size = 32, verbose = 2, shuffle = True, class_weight = {0 : 0.294,
                                                                                                  1 : 0.330,
                                                                                                  2 : 0.172,
                                                                                                  3 : 0.204},
         validation_split = 0.2)

Epoch 1/50
124/124 - 1s - loss: 0.1587 - mean_absolute_error: 0.5028 - val_loss: 0.3478 - val_mean_absolute_error: 0.3478
Epoch 2/50
124/124 - 1s - loss: 0.0667 - mean_absolute_error: 0.2140 - val_loss: 0.3031 - val_mean_absolute_error: 0.3031
Epoch 3/50
124/124 - 1s - loss: 0.0457 - mean_absolute_error: 0.1483 - val_loss: 0.2674 - val_mean_absolute_error: 0.2674
Epoch 4/50
124/124 - 1s - loss: 0.0379 - mean_absolute_error: 0.1233 - val_loss: 0.2504 - val_mean_absolute_error: 0.2504
Epoch 5/50
124/124 - 1s - loss: 0.0326 - mean_absolute_error: 0.1061 - val_loss: 0.2447 - val_mean_absolute_error: 0.2447
Epoch 6/50
124/124 - 1s - loss: 0.0290 - mean_absolute_error: 0.0943 - val_loss: 0.2363 - val_mean_absolute_error: 0.2363
Epoch 7/50
124/124 - 1s - loss: 0.0268 - mean_absolute_error: 0.0871 - val_loss: 0.2358 - val_mean_absolute_error: 0.2358
Epoch 8/50
124/124 - 1s - loss: 0.0240 - mean_absolute_error: 0.0780 - val_loss: 0.2274 - val_mean_absolute_error: 0.2274
Epoch 9/50
124/124 - 1s 

KeyboardInterrupt: ignored